In [1]:
!pip install flask
!pip install pyngrok
!pip install flask_cors

import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from flask import Flask,request,jsonify
from pyngrok import ngrok
from flask_cors import CORS
# Load dataset
data = pd.read_csv('/content/Coursera.csv')  # Replace with actual file path

# Preprocessing
data['Skills'] = data['Skills'].fillna('')
data['Combined_Text'] = data['Course Description'] + " " + data['Skills']

# Vectorization
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(data['Combined_Text'])

# Recommendation Function
def recommend_courses(interest, skills, level):
    query = interest + " " + skills
    query_vector = tfidf.transform([query])

    # Cosine Similarity
    similarity = cosine_similarity(query_vector, tfidf_matrix)

    # Add similarity scores
    data['Similarity'] = similarity[0]

    # Filter by difficulty level
    filtered_data = data[data['Difficulty Level'].str.lower() == level.lower()]

    # Sort only by Similarity
    results = filtered_data.sort_values(['Similarity'], ascending=False)

    # Return Top 5 Recommendations
    return results[['Course Name', 'University', 'Course Rating', 'Similarity', 'Course URL','Course Description']].head(5)

app = Flask(__name__)
CORS(app)
@app.route('/recommend',methods=['POST'])


def recommend():
    # Get the input data from the POST request
    try:
        req_data = request.get_json()
        interest = req_data['interest']
        skills = req_data['skills']
        level = req_data['level']
    except KeyError:
        return jsonify({'error': 'Missing required fields'}), 400

    # Get recommendations
    recommendations = recommend_courses(interest, skills, level)

    # Convert recommendations to dictionary format
    recommendations_list = recommendations.to_dict(orient='records')

    return jsonify(recommendations_list)

# Start Flask Server
if __name__ == '__main__':
    # Create a tunnel to the Flask app through ngrok
    ngrok.set_auth_token('2rA9QT8w4GXftuLHB7rLNG5R8vM_6NbtYDUYp55c4bLwthA79')
    public_url = ngrok.connect(5000)
    print(f"Flask app is live at {public_url}")
    app.run(port=5000)

# Take Input from User
# interest = input("Enter your area of interest: ").strip()
# skills = input("Enter your current skills (comma-separated): ").strip()
# level = input("Enter your difficulty level (Beginner, Intermediate, Advanced): ").strip()

# # Display Recommendations
# recommendations = recommend_courses(interest, skills, level)
# print("\nTop Course Recommendations:")
# print(recommendations.to_string(index=False))


Flask app is live at NgrokTunnel: "https://d9d2-35-185-48-155.ngrok-free.app" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [12/Jan/2025 13:16:52] "POST /recommend HTTP/1.1" 200 -


In [1]:
!pip install pyngrok
!pip install flask_cors